# Vendas de Jogos de Videogames

In [7]:
import numpy as np
import pandas as pd
import scipy.stats as st
pd.set_option('display.max_columns', None)

import math

import matplotlib.pyplot as plt
%matplotlib inline    #faz com que os gráficos gerados sejam mostrados dentro do notebook
plt.style.use('ggplot')   #estilo que os gráficos terão. Adiciona cores e padrões diferentes para personalizar

import seaborn as sns
sns.set_style('whitegrid')

import missingno as msno

from sklearn.preprocessing import StandardScale
from scipy import stats

# import os 
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


UsageError: unrecognized arguments: #faz com que os gráficos gerados sejam mostrados dentro do notebook


In [10]:
# Leitura do arquivo
videogames = pd.read_csv('dados/vgsales.csv')
videogames

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


In [11]:
# Renomeando as colunas
videogames.columns = ['Ranking', 'Nome', 'Plataforma', 'Ano', 'Gênero', 'Editora', 'Vendas América do Norte',
                      'Vendas EUA', 'Vendas Japão', 'Outras Vendas', 'Vendas Global']
videogames.head(10)

,Ranking,Nome,Plataforma,Ano,Gênero,Editora,Vendas América do Norte,Vendas EUA,Vendas Japão,Outras Vendas,Vendas Global
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
5,6,Tetris,GB,1989.0,Puzzle,Nintendo,23.20,2.26,4.22,0.58,30.26
6,7,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.38,9.23,6.50,2.90,30.01
7,8,Wii Play,Wii,2006.0,Misc,Nintendo,14.03,9.20,2.93,2.85,29.02
8,9,New Super Mario Bros. Wii,Wii,2009.0,Platform,Nintendo,14.59,7.06,4.70,2.26,28.62
9,10,Duck Hunt,NES,1984.0,Shooter,Nintendo,26.93,0.63,0.28,0.47,28.31


In [13]:
drop_row_index = videogames[videogames['Ano'] > 2015].index
videogames = videogames.drop(drop_row_index)
videogames

,Ranking,Nome,Plataforma,Ano,Gênero,Editora,Vendas América do Norte,Vendas EUA,Vendas Japão,Outras Vendas,Vendas Global
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


In [14]:
# Quantidade de linhas e colunas no Dataframe
videogames.shape

(16250, 11)

In [15]:
# Resumo de informações
videogames.describe()

,Ranking,Ano,Vendas América do Norte,Vendas EUA,Vendas Japão,Outras Vendas,Vendas Global
count,16250.000000,15979.000000,16250.000000,16250.000000,16250.000000,16250.000000,16250.000000
mean,8233.153785,2006.197071,0.268924,0.148146,0.078601,0.048614,0.544564
std,4775.382512,5.714810,0.824467,0.509035,0.312196,0.190271,1.569213
min,1.000000,1980.000000,0.000000,0.000000,0.000000,0.000000,0.010000
25%,4095.250000,2003.000000,0.000000,0.000000,0.000000,0.000000,0.060000
50%,8213.500000,2007.000000,0.080000,0.020000,0.000000,0.010000,0.170000
75%,12340.750000,2010.000000,0.240000,0.110000,0.040000,0.040000,0.480000
max,16600.000000,2015.000000,41.490000,29.020000,10.220000,10.570000,82.740000


In [ ]:
# Tipos de dados em cada coluna
videogames.dtypes

In [ ]:
# Verificando linhas com NaN
videogames[videogames['Ano'].isnull()]

In [ ]:
# Verificando quantidade de jogos por plataforma
videogames['Plataforma'].value_counts()

In [2]:
# videogames['Plataforma'].value_counts().plot()

## Criando gráficos

In [3]:
# Os 10 videogames com mais títulos lançados
videogames['Plataforma'].value_counts().head(10).plot(kind = 'bar', figsize = (11,5), grid = False, color = 'green')

plt.title('Os 10 videogames com mais títulos lançados')  # Adicionando um título ao gráfico
plt.xlabel('Videogames')  # Renomeando eixo x
plt.ylabel('Quantidade de jogos lançados')  # Renomeando eixo y
plt.show()

NameError: name 'videogames' is not defined

In [ ]:
# Top 10 - os 10 jogos mais vendidos
tp_10_vendidos = videogames[['Nome', 'Vendas Global']].head(10).set_index('Nome').sort_values('Vendas Global', ascending = True)
tp_10_vendidos.plot(kind = 'barh', figsize = (11,7), grid = True, color = 'darkred', legend = False )

plt.title('Os 10 jogos mais vendidos')
plt.xlabel('Total de vendas (em milhões de dólares)')
plt.show()

# Informações cruzadas

In [ ]:
crosstab_vg = pd.crosstab(videogames['Plataforma'], videogames['Gênero'])
crosstab_vg.head()

In [ ]:
# Acrescentando o total 
crosstab_vg['Total'] = crosstab_vg.sum(axis = 1)
crosstab_vg.head()

## Heatmap - gráfico de calor

In [ ]:
#crio um DataFrame pra armazenar os 10 consoles que tiveram mais ançamentos, coloco uma condição > 1000 e utilizo os 
#sort_values para ordenar pela coluna total em ordem decrescente
top10_plataforma = crosstab_vg[crosstab_vg['Total'] > 1000].sort_values('Total', ascending = False) 

# aqui crio um novo DataFrame similar ao anterior, mas que adiciona uma nova linha com o total no final do DataFrame
top10_final = top10_plataforma.append(pd.DataFrame(top10_plataforma.sum(), columns = ['Total']).T, ignore_index = False)

sns.set(font_scale = 1) #tamanho da fonte no gráfico
plt.figure(figsize = (18,9)) #tamanho do gráfico

#utilizo a função heatmap (que cria o mapa de calor). O primeiro argumento é o que quero exibir, no caso top10_final, 
#depois annot faz com que os números dos quadrados apareçam. Após vmax e vmim definem os valores máximos e mínimos, para isso 
#utilizei a função .loc e filtro as linhas e colunas do DataFrama. Por fim, fmt para formatar as informações pra ficarem bem apresentadas
sns.heatmap(top10_final, annot = True, vmax = top10_final.loc[: 'PS', : 'Strategy'].values.max(), 
            vmin = top10_final.loc[: , : 'Strategy'].values.min(), fmt = 'd' )
plt.xlabel('GÊNERO')
plt.ylabel('CONSOLE')
plt.title('QUANTIDADE DE TÍTULOS POR GÊNERO E CONSOLE')
plt.show()